In [144]:
import numpy as np
import pandas as pd
import os
import re
from statistics import mean

In [195]:
#FASTA file parser code

file="FinalProject.fasta"

f=open(file)
sequence_names=[]
sequences=[]
count=0
seq=""

for line in f:
    if line.startswith(">"):
        sequence_names.append(line[1:].strip()) #to remove the > and \n
        count=count+1
        sequences.append("") #initializing the slot for the sequence
    elif not line.startswith('#') and line!="":
        sequences[count-1]=sequences[count-1]+line.strip().replace('-',"")

f.close()

#initialize data
data = {'ID':sequence_names,
        'Sequence':sequences}
  
#dataframe
dfseq = pd.DataFrame(data)

#print
dfseq
dfseq=dfseq.set_index('ID')

In [196]:
#number of sequences in file
len(dfseq)

6

In [197]:
#extracting primers, introns, and exons, and storing then in pandas dataframe
primers=[]
exons=[]
introns=[]

for i in range(0, len(dfseq)):
    p=re.findall('[a-z]+', dfseq.Sequence[i])[0] #the first lowercase is the primer
    primers.append(p)
    
    ex=re.findall('[A-Z]+', dfseq.Sequence[i]) #all uppercases are exons
    exons.append(ex)
    
    intr=re.findall('[a-z]+', dfseq.Sequence[i])[1:-1] #all lowercases are introns: exclude upperstream and lowerstream
    introns.append(intr)
    
#adding primer, exons and introns to the dataframe
dfseq['Primer']=primers
dfseq['Exons']=exons
dfseq['Introns']=introns

#dataframe with seq ID and all its corresponding information
dfseq

,Sequence,Primer,Exons,Introns
ID,,,,
HCN1,gccgaggggaggcgctggggcgcgagggctGAGCCGAGCTGAGCCT...,gccgaggggaggcgctggggcgcgagggct,[GAGCCGAGCTGAGCCTCGCCTAGCTCCGGCAGCCTCAGCTTCAGC...,[gtgagggctcccggtcgccgccccaccctcccttcaggcgcgccc...
HCN2,ccgccccgccccgcccccgcctcccccctCCCTCGGGCTCCGGCCG...,ccgccccgccccgcccccgcctcccccct,[CCCTCGGGCTCCGGCCGGCGGCGGCGGCGGCGGCTCCGCTCCGCA...,[gtaccgcctccgggagggccggtcggcgcgagggggcccggggag...
KCNA1,gggctgcgcttcgagacgcagctcaagaccCTGGCGCAGTTCCCCA...,gggctgcgcttcgagacgcagctcaagacc,[CTGGCGCAGTTCCCCAACACGCTGCTGGGCAACCCTAAGAAACGC...,[gtaagcatgtttgaatctgatacaatttattttataatcgcatgc...
KCNA2,agcgctgaccaagaaaggaagtggtgatggGGCACATAGAAGAGTG...,agcgctgaccaagaaaggaagtggtgatgg,[GGCACATAGAAGAGTGAGCCATCATCTGGTTTCCAGCGCCAAGAC...,[gtgagagaagaggaggggcagcaagggtgagcatgcatgtgtgga...
KCNS3,cctcgctctagcggggcgggaccgacggacAGACCGGCCGACGCGG...,cctcgctctagcggggcgggaccgacggac,[AGACCGGCCGACGCGGGCCACCCCGCTCTCCTCGCCGCCGCGGCG...,[gtacctggctggcgtccccacctccctgcgcgctccggagacttc...
KIR3,gcgcggccgcctgtctgcaccggcagcaccATGTCGCTCATGGTCG...,gcgcggccgcctgtctgcaccggcagcacc,"[ATGTCGCTCATGGTCGTCAGCATGGCGTGTGTTG, GAGTCCACA...",[gtgagtcctggaaaggaatagagggagggagtgccacatcctcct...


In [220]:
#number of introns
dfseq['Introns'].apply(len)

ID
HCN1     7
HCN2     7
KCNA1    1
KCNA2    4
KCNS3    2
KIR3     2
Name: Introns, dtype: int64

In [199]:
#number of exons
dfseq['Exons'].apply(len)

ID
HCN1     8
HCN2     8
KCNA1    2
KCNA2    5
KCNS3    3
KIR3     3
Name: Exons, dtype: int64

In [200]:
#function that returns average string length in a list of strings
#uses mean from statistics package

def avgstrlen(list):
    return mean([len(i) for i in list])

In [201]:
#average intron length per sequence
dfseq['Introns'].apply(avgstrlen)

ID
HCN1     61642.857143
HCN2      3408.428571
KCNA1     4062.000000
KCNA2     8984.000000
KCNS3    25975.500000
KIR3      1668.000000
Name: Introns, dtype: float64

In [202]:
#average exon length per sequence
dfseq['Exons'].apply(avgstrlen)

ID
HCN1     1241.625000
HCN2      427.500000
KCNA1    1309.000000
KCNA2     391.800000
KCNS3     780.333333
KIR3      209.333333
Name: Exons, dtype: float64

In [203]:
#function that calculates the percentage occurence of a letter in a list of strings

def occurencepercentage(letter, list):
    str=''.join(list).upper() #case insensitive, join list elements into a single string
    freq=str.count(letter.upper()) #occurence of letter in string
    perc=100*(float(freq)/float(len(str))) #percentage
    return perc

#test
#ls=["hi","hey","heee"]
#occurencepercentage('h', ls)

#functions that calculate percentage occurence of each nucleotide (to be used with apply)
def occurenceA(list):
    return occurencepercentage('A', list)

def occurenceG(list):
    return occurencepercentage('G', list)

def occurenceC(list):
    return occurencepercentage('C', list)

def occurenceT(list):
    return occurencepercentage('T', list)

In [207]:
d=dfseq['Introns'].apply(occurenceC)
d

ID
HCN1     16.936732
HCN2     28.710340
KCNA1    20.827179
KCNA2    23.216273
KCNS3    19.254682
KIR3     20.233813
Name: Introns, dtype: float64

In [208]:
d[0]

16.936732329084588

In [226]:
#sequence IDS
list(dfseq.index)

['HCN1', 'HCN2', 'KCNA1', 'KCNA2', 'KCNS3', 'KIR3']

In [232]:
#number of exons
dfseq['NumExons']=dfseq['Exons'].apply(len).tolist()

#number of introns
dfseq['NumIntrons']=dfseq['Introns'].apply(len).tolist()

#AvgExonLength
dfseq['AvgExonLength']=dfseq['Exons'].apply(avgstrlen).tolist()
#AvgIntronLength
dfseq['AvgIntronLength']=dfseq['Introns'].apply(avgstrlen).tolist()

#%A_Exon
dfseq['%A_Exon']=dfseq['Exons'].apply(occurenceA).tolist()
#%C_Exon
dfseq['%C_Exon']=dfseq['Exons'].apply(occurenceC).tolist()
#%G_Exon
dfseq['%G_Exon']=dfseq['Exons'].apply(occurenceG).tolist()
#%T_Exon
dfseq['%T_Exon']=dfseq['Exons'].apply(occurenceT).tolist()

#%A_Intron
dfseq['%A_Intron']=dfseq['Introns'].apply(occurenceA).tolist()
#%C_Intron
dfseq['%C_Intron']=dfseq['Introns'].apply(occurenceC).tolist()
#%G_Intron
dfseq['%G_Intron']=dfseq['Introns'].apply(occurenceG).tolist()
#%T_Intron
dfseq['%T_Intron']=dfseq['Introns'].apply(occurenceT).tolist()

In [233]:
dfseq

,Sequence,Primer,Exons,Introns,NumExons,NumIntrons,AvgExonLength,AvgIntronLength,%A_Exon,%C_Exon,%G_Exon,%T_Exon,%A_Intron,%C_Intron,%G_Intron,%T_Intron
ID,,,,,,,,,,,,,,,,
HCN1,gccgaggggaggcgctggggcgcgagggctGAGCCGAGCTGAGCCT...,gccgaggggaggcgctggggcgcgagggct,[GAGCCGAGCTGAGCCTCGCCTAGCTCCGGCAGCCTCAGCTTCAGC...,[gtgagggctcccggtcgccgccccaccctcccttcaggcgcgccc...,8,7,1241.625000,61642.857143,30.252693,20.215443,20.004027,29.527837,32.098494,16.936732,17.690151,33.274623
HCN2,ccgccccgccccgcccccgcctcccccctCCCTCGGGCTCCGGCCG...,ccgccccgccccgcccccgcctcccccct,[CCCTCGGGCTCCGGCCGGCGGCGGCGGCGGCGGCTCCGCTCCGCA...,[gtaccgcctccgggagggccggtcggcgcgagggggcccggggag...,8,7,427.500000,3408.428571,15.116959,38.713450,32.105263,14.064327,17.527977,28.710340,33.345907,20.415776
KCNA1,gggctgcgcttcgagacgcagctcaagaccCTGGCGCAGTTCCCCA...,gggctgcgcttcgagacgcagctcaagacc,[CTGGCGCAGTTCCCCAACACGCTGCTGGGCAACCCTAAGAAACGC...,[gtaagcatgtttgaatctgatacaatttattttataatcgcatgc...,2,1,1309.000000,4062.000000,27.731092,24.407945,21.275783,26.585180,28.385032,20.827179,20.704087,30.083703
KCNA2,agcgctgaccaagaaaggaagtggtgatggGGCACATAGAAGAGTG...,agcgctgaccaagaaaggaagtggtgatgg,[GGCACATAGAAGAGTGAGCCATCATCTGGTTTCCAGCGCCAAGAC...,[gtgagagaagaggaggggcagcaagggtgagcatgcatgtgtgga...,5,4,391.800000,8984.000000,24.400204,26.901480,24.196018,24.502297,25.486977,23.216273,23.413847,27.882903
KCNS3,cctcgctctagcggggcgggaccgacggacAGACCGGCCGACGCGG...,cctcgctctagcggggcgggaccgacggac,[AGACCGGCCGACGCGGGCCACCCCGCTCTCCTCGCCGCCGCGGCG...,[gtacctggctggcgtccccacctccctgcgcgctccggagacttc...,3,2,780.333333,25975.500000,23.878684,23.964118,25.544639,26.612559,25.373910,19.254682,21.911032,33.460376
KIR3,gcgcggccgcctgtctgcaccggcagcaccATGTCGCTCATGGTCG...,gcgcggccgcctgtctgcaccggcagcacc,"[ATGTCGCTCATGGTCGTCAGCATGGCGTGTGTTG, GAGTCCACA...",[gtgagtcctggaaaggaatagagggagggagtgccacatcctcct...,3,2,209.333333,1668.000000,20.541401,30.095541,25.000000,24.363057,32.224221,20.233813,27.607914,19.934053


In [235]:
dfseqstat=dfseq[['NumExons','NumIntrons', 'AvgExonLength', 'AvgIntronLength', '%A_Exon', '%C_Exon', '%G_Exon', '%T_Exon', '%A_Intron', '%C_Intron','%G_Intron','%T_Intron']]

In [245]:
dfseqstat=dfseqstat.round()

In [250]:
#Write to DNAStats.txt
filename=file+'DNAStats.txt'
dfseqstat.to_csv(filename, header=True, index=True, sep='\t')

In [251]:
print(dfseqstat)

       NumExons  NumIntrons  AvgExonLength  AvgIntronLength  %A_Exon  %C_Exon  \
ID                                                                              
HCN1          8           7         1242.0          61643.0     30.0     20.0   
HCN2          8           7          428.0           3408.0     15.0     39.0   
KCNA1         2           1         1309.0           4062.0     28.0     24.0   
KCNA2         5           4          392.0           8984.0     24.0     27.0   
KCNS3         3           2          780.0          25976.0     24.0     24.0   
KIR3          3           2          209.0           1668.0     21.0     30.0   

       %G_Exon  %T_Exon  %A_Intron  %C_Intron  %G_Intron  %T_Intron  
ID                                                                   
HCN1      20.0     30.0       32.0       17.0       18.0       33.0  
HCN2      32.0     14.0       18.0       29.0       33.0       20.0  
KCNA1     21.0     27.0       28.0       21.0       21.0       30.0  
K

In [ ]:
#SINGLE FUNCTION THAT DOES ALL THE ABOVE

